In [96]:
from openai import OpenAI

In [97]:
!head documents.json

[
  {
    "course": "data-engineering-zoomcamp",
    "documents": [
      {
        "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
        "section": "General course-related questions",
        "question": "Course - When will the course start?"
      },
      {


In [98]:
import json

# flattening the doc
with open('/workspaces/projectLLM/documents.json', 'rt') as f_in:
    document_file = json.load(f_in)

documents = []

for course in document_file:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [99]:
documents[2]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [100]:
len(documents)

948

In [101]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
es.info()

ObjectApiResponse({'name': '83b49a6006b8', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'dP7rdlZxRNe5IyN9sUskQQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [102]:
index_setting = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
response = es.indices.create(index=index_name, body=index_setting)
response

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/UQmx-q5VT72QtCMW4bO68A] already exists')

In [ ]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es.index(index=index_name, body=index_setting)

  0%|          | 0/948 [00:00<?, ?it/s]

In [ ]:
user_question = 'Course - Can I still join the course after the start date?'

In [ ]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [ ]:
response = es.search(index=index_name, body=search_query)

In [ ]:
response

ObjectApiResponse({'took': 4, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}})

In [ ]:
result_docs = []

for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [ ]:
result_docs

[]

In [ ]:
l = 5
for i in range(l):
    for j in range(i):
        print('*',end="")
    print("")


*
**
***
****
